# 2. Gadget decomposition

**GOAL:** for a given polynomial $\boldsymbol{a}$ and an encryption $\textsf{RLWE}(m)$ (or something similar to it), find $\textsf{RLWE}(\boldsymbol{a}\cdot \boldsymbol{m})$, with small noise.

Gadget decomposition and $RLWE'$ technique allows us to multiply ciphertext to a *large* constant with *small* noise increment.

RLWE' is usually used for the key switching. We make an example of key switching from s1 to s2 here.

In [35]:
# Functions from previous lecturenote
import torch
import math

stddev = 3.2
logQ = 27

N = 2**10
Q = 2**logQ


def keygen(dim):
    return torch.randint(2, size = (dim,))

def errgen(stddev):
    e = torch.round(stddev*torch.randn(1))
    e = e.squeeze()
    return e.to(torch.int)

def errgen(stddev, N):
    e = torch.round(stddev*torch.randn(N))
    e = e.squeeze()
    return e.to(torch.int)

def uniform(dim, modulus):
    return torch.randint(modulus, size = (dim,))

def polymult(a, b, dim, modulus):
    res = torch.zeros(dim).to(torch.int)
    for i in range(dim):
        for j in range(dim):
            if i >= j:
                res[i] += a[j]*b[i-j]
                res[i] %= modulus
            else:
                res[i] -= a[j]*b[i-j] # Q - x mod Q = -x
                res[i] %= modulus

    res %= modulus
    return res

root_powers = torch.arange(N//2).to(torch.complex128)
root_powers = torch.exp((1j*math.pi/N)*root_powers)

root_powers_inv = torch.arange(0,-N//2,-1).to(torch.complex128)
root_powers_inv = torch.exp((1j*math.pi/N)*root_powers_inv)

def negacyclic_fft(a, N, Q):
    acomplex = a.to(torch.complex128)

    a_precomp = (acomplex[...,:N//2] + 1j * acomplex[..., N//2:]) * root_powers

    return torch.fft.fft(a_precomp)

def negacyclic_ifft(A, N, Q):
    b = torch.fft.ifft(A)
    b *= root_powers_inv

    a = torch.cat((b.real, b.imag), dim=-1)

    aint = a.to(torch.int32)
    # only when Q is a power-of-two
    aint &= Q-1

    return aint

In [36]:
# generate two keys
s1 = keygen(N)
s2 = keygen(N)

s1fft = negacyclic_fft(s1, N, Q)
s2fft = negacyclic_fft(s2, N, Q)

In [252]:
# make an RLWE encryption of message
def encrypt_to_fft(m, sfft):
    ct = torch.stack([errgen(stddev, N), uniform(N, Q)])
    ctfft = negacyclic_fft(ct, N, Q)

    ctfft[0] += -ctfft[1]*sfft + negacyclic_fft(m, N, Q)

    return ctfft

def normalize(v, logQ):
    # same as follows but no branch
    """
    if v > Q//2:
        v -= Q
    """
    # vmod Q when Q is a power-of-two
    Q = (1 << logQ)
    v &= Q-1
    # get msb
    msb = (v & Q//2) >> (logQ - 1)
    v -= (Q) * msb
    return v

def decrypt_from_fft(ctfft, sfft):
    return normalize(negacyclic_ifft(ctfft[0] + ctfft[1]*sfft, N, Q), logQ)

In [79]:
m = torch.zeros((N), dtype=torch.int32)
m[0] = 1000000

m

tensor([1000000,       0,       0,  ...,       0,       0,       0],
       dtype=torch.int32)

In [39]:
ctfft = encrypt_to_fft(m, s1fft)
mdec = decrypt_from_fft(ctfft, s1fft)
mdec

tensor([1000003,       0,       1,  ...,       3,      -4,       1],
       dtype=torch.int32)

We cannot decrypt mdec with s2, as the secret key is different. It should look like a random value.

In [40]:
mdec_wrong = decrypt_from_fft(ctfft, s2fft)
mdec_wrong

tensor([ 11712410,  49146338,  22304380,  ..., -67061463,  44753809,
         -3073699], dtype=torch.int32)

We want to transform ct as a ciphertext of the same message but with different key s2 *without decryption*.

## 2.1. Gadget decomposition

We define *gadget decomposition* $h$ corresponding to gadget vector $\vec{g} = (g_0, g_1, \dots, g_{d-1})$ as follows.
$$
h: \mathbb{Z} \longmapsto \mathbb{Z}^d
$$
$$
||h(a)|| < B, \left< h(a), \vec{g}\right> = a,
$$
where B is a upper bound.

Also, we can naturally extend it to $\mathcal{R}$ and $\mathbb{Z}^n$.

For example, a number $77 = 0\text{b}01001101$ can be decomposed to $(1,0,1,1,0,0,1,0)$ when $\vec{g} = (1, 2, \dots, 2^7)$ and $B=2$

Otherwise, it can also be decomposed to $(1,3,0,1)$ when $\vec{g} = (1, 4, 4^2, 4^3)$, and $B=4$ here.

We also can do a *approximate gadget decomposition* where $\left< h(a), \vec{g}\right> \approx a$, i.e., $\left< h(a), \vec{g}\right>$ is not exactly the same but similar to $a$.

We first set d and B satisfying $B^{d} \le Q \le B^{d+1}$

In [224]:
# we will use B-ary decomposition, i.e., cut digits by logB bits
d = 4
logB = 6

We shift by amount of decomp_shit and cut `logB` MSBs

In [225]:
decomp_shift = logQ - logB*torch.arange(d,0,-1).view(d,1)
decomp_shift

mask = (1 << logB) - 1

decomp_shift

tensor([[ 3],
        [ 9],
        [15],
        [21]])

We can see that the gadget vector, 
$$
\vec{g} = (B, B^2, \dots, B^{d-1})
$$
is given as follows.

In [226]:
gvector = 1 << decomp_shift
gvector

tensor([[      8],
        [    512],
        [  32768],
        [2097152]])

The decomposition function handles both RLWE' and RGSW case.

In [231]:
def decompose(a):
    
    assert len(a.size()) <= 2
    # for RLWE'
    if len(a.size()) == 1:
        res = (a.unsqueeze(0) >> decomp_shift.view(d, 1)) & mask
        return res
    # for RGSW
    elif len(a.size()) == 2:
        res = (a.unsqueeze(0) >> decomp_shift.view(d, 1, 1)) & mask
        return res



In [241]:
a = uniform(N, Q)
a[:10]

tensor([112837406,  94435928,   2684047,  86305079, 128185452, 106015573,
        105584985,  81841074,  18582303,  64143918])

See the decomposed vector always has value less than $B = 2^6 = 64$.

In [242]:
da = decompose(a)
# see all values are smaller than 2^6 = 64
da

tensor([[35, 11, 17,  ...,  2,  9, 53],
        [33, 61, 58,  ...,  5,  1, 43],
        [51,  1, 17,  ..., 58, 35, 36],
        [53, 45,  1,  ..., 39, 28, 57]])

See $<h(a), \vec{g}> \approx a$. 

In [243]:
#composition is inner product, see it is similar to a
torch.sum(da * gvector, dim = 0)

tensor([112837400,  94435928,   2684040,  ...,  83692048,  59867720,
        120739752])

In [244]:
a

tensor([112837406,  94435928,   2684047,  ...,  83692050,  59867721,
        120739755])

In [245]:
diff = a - torch.sum(da * gvector, dim = 0)
diff %= Q
diff = normalize(diff, logQ)
diff[:10]

tensor([6, 0, 7, 7, 4, 5, 1, 2, 7, 6])

We can extend it to a ciphertext too

In [249]:
m = torch.zeros(N)
m[0] = Q//4
m[3] = Q//4
print("m:\n", m[:10])
ctfft = encrypt_to_fft(m, s1fft)
ct = negacyclic_ifft(ctfft, N, Q)
ct

m:
 tensor([33554432.,        0.,        0., 33554432.,        0.,        0.,
               0.,        0.,        0.,        0.])


tensor([[ 86118725,  22323987,  42856786,  ..., 110045621,  58693276,
          28320585],
        [123551445, 104439256, 100377657,  ..., 104293947, 116006042,
          95227960]], dtype=torch.int32)

We will see the message part in the next subsection

In [250]:
ctnew = torch.sum(decompose(ct) * gvector.view(d, 1, 1), dim = 0)
ctnew

tensor([[ 86118720,  22323984,  42856784,  ..., 110045616,  58693272,
          28320584],
        [123551440, 104439256, 100377656,  ..., 104293944, 116006040,
          95227960]])

Decrypt it

In [251]:
ctnewfft = negacyclic_fft(ctnew, N, Q)

dm = decrypt_from_fft(ctnewfft, s1fft)[:10]
print("m:\n", m[:10])
print("dm:\n", dm[:10])


RuntimeError: Overflow when unpacking long

## 2.2.RLWE' Ciphertext and key switching keys.

We can make a tuple of RLWE ciphertexts corresponding to a gadget vector $\vec{g} = (g_0, \dots, g_{d-1})$, and call it RLWE'.
$$
RLWE'( \boldsymbol{s} ) =\left( RLWE(g_0 \boldsymbol{s}), RLWE(g_1 \boldsymbol{s}), \dots, RLWE(g_{d-1} \boldsymbol{s})  \right) 
\in \mathcal{R}^{d\times N}
$$

Then the inner product between $h(\boldsymbol{m})$ and $RLWE'( \boldsymbol{s} )$ will give us RLWE (not RLWE'!!!) encryption of $\boldsymbol{m\cdot s}$, $RLWE(\boldsymbol{m \cdot s})$.
The correctness can be seen as follows:
$$
\left<(\boldsymbol{m}_0, \dots, \boldsymbol{m}_{d-1}),  \left( RLWE(g_0 \boldsymbol{s}), \dots, RLWE(g_{d-1} \boldsymbol{s})  \right) \right>
= \sum_{i = 0}^{d-1} (\boldsymbol{m}_i \cdot RLWE(g_i \boldsymbol{s}))
= RLWE( \sum_{i = 0}^{d-1} (\boldsymbol{m}_i \cdot g_i \boldsymbol{s}))
= RLWE( \sum_{i = 0}^{d-1} (\boldsymbol{m} \cdot \boldsymbol{s}))
$$



### 2.2.1. Error analysis (why RLWE'?)

We can also get a ciphertext of $RLWE(\boldsymbol{m}\boldsymbol{s})$ by multiplying $\boldsymbol{m}$ to each element  of $RLWE(\boldsymbol{s})$.
In other words, $(\boldsymbol{m} \cdot \boldsymbol{b}, \boldsymbol{m}\cdot \boldsymbol{a})$ is $RLWE(\boldsymbol{m}\boldsymbol{s})$, 
where $RLWE(\boldsymbol{s}) = (\boldsymbol{b}, \boldsymbol{a})$.

**Naive multiplication**

However, an error $\boldsymbol{e}$ is contained in $RLWE(\boldsymbol{s})$, so the decryption $\boldsymbol{b} + \boldsymbol{a} \cdot  \boldsymbol{s}$ will be given as 
$$ 
\boldsymbol{s} +  \boldsymbol{e}.
$$
Thus, the decryption of $RLWE(\boldsymbol{ms}) = (\boldsymbol{mb}, \boldsymbol{ma})$ results in $ \boldsymbol{ms} +  \boldsymbol{me}$.

**Multiplication using RLWE'**

It is okay when $\boldsymbol{m}$ is small (so it is used to multiply a small constant), but we usually need to to multiply $\boldsymbol{m}$ uniformly sampled in $\mathbb{Z}_Q$.
In this case, the error variance will be $O(Q^2 \sigma^2)$, where $\sigma$ is variance of $\boldsymbol{e}$, and it overwhelms the message.

Instead, if we use the RLWE' product, each ciphertext $RLWE'(g_i \boldsymbol{s})$ is multiplied by $\boldsymbol{m}_i$, whose size is smaller than $B$.
Assuming they are uniformly distributed, the error variance of $RLWE'(\boldsymbol{m}_i g_i \boldsymbol{s})$ should be $B^2/12 \sigma^2$.
Adding $d$ of them, the error variance is  
$$
dB^2/12 \sigma^2,
$$ 
where $B = Q^{1/d} \ll Q$.

Naive multiplication is *infeasible* when $\boldsymbol{m}$ is large, which is exactly the case we want. 
So, we need RLWE' ciphertext (as key) and multiplication using it.

Let's encrypt a key using RLWE' ciphertext.

In [ ]:
# it has a dimension of d, 2, N
rlwep = torch.zeros(d, 2, N, dtype=torch.int32)

# generate the 'a' part
rlwep[:, 1, :] = torch.randint(Q, size = (d, N), dtype= torch.int32)

rlwep

tensor([[[        0,         0,         0,  ...,         0,         0,
                  0],
         [  1788411,  62139972,  70842809,  ...,   7371080,  14571503,
           41590556]],

        [[        0,         0,         0,  ...,         0,         0,
                  0],
         [ 32409271, 120764302,   5680244,  ...,  24170642,  95132708,
           56714077]],

        [[        0,         0,         0,  ...,         0,         0,
                  0],
         [ 34838124,  76878026,   2307585,  ..., 100510751,  71217482,
          110731723]],

        [[        0,         0,         0,  ...,         0,         0,
                  0],
         [ 34404428,  12475759, 118260867,  ..., 126818329,  10023518,
          114571333]]], dtype=torch.int32)

In [ ]:
# add error on b
rlwep[:, 0, :] = torch.round(stddev * torch.randn(size = (d, N)))

rlwep

tensor([[[       -6,         1,        -9,  ...,        -2,        -3,
                  3],
         [  1788411,  62139972,  70842809,  ...,   7371080,  14571503,
           41590556]],

        [[        0,        -2,         3,  ...,        -4,        -1,
                 -3],
         [ 32409271, 120764302,   5680244,  ...,  24170642,  95132708,
           56714077]],

        [[       -6,         2,         2,  ...,        -2,         2,
                  1],
         [ 34838124,  76878026,   2307585,  ..., 100510751,  71217482,
          110731723]],

        [[       -1,        -6,        -5,  ...,        -4,        -1,
                 -2],
         [ 34404428,  12475759, 118260867,  ..., 126818329,  10023518,
          114571333]]], dtype=torch.int32)

In [ ]:
# following is equal to rlwep %= Q, but a faster version
rlwep &= (Q-1)

rlwep

tensor([[[134217722,         1, 134217719,  ..., 134217726, 134217725,
                  3],
         [  1788411,  62139972,  70842809,  ...,   7371080,  14571503,
           41590556]],

        [[        0, 134217726,         3,  ..., 134217724, 134217727,
          134217725],
         [ 32409271, 120764302,   5680244,  ...,  24170642,  95132708,
           56714077]],

        [[134217722,         2,         2,  ..., 134217726,         2,
                  1],
         [ 34838124,  76878026,   2307585,  ..., 100510751,  71217482,
          110731723]],

        [[134217727, 134217722, 134217723,  ..., 134217724, 134217727,
          134217726],
         [ 34404428,  12475759, 118260867,  ..., 126818329,  10023518,
          114571333]]], dtype=torch.int32)

In [ ]:
# do fft for easy a*s
rlwepfft = negacyclic_fft(rlwep, N, Q)

# now b = -a*s2 + e
rlwepfft[:, 0, :] -= rlwepfft[:, 1, :] * s2fft.view(1,N//2)

In [ ]:
# return back to R_Q
rlwepifft = negacyclic_ifft(rlwepfft, N, Q)

rlwepifft

tensor([[[ 50119422,  47955367,  26405086,  ...,  34553414,   2621741,
          119429386],
         [  1788410,  62139972,  70842808,  ...,   7371080,  14571503,
           41590556]],

        [[ 54748284,  51041966, 122316881,  ..., 128565749, 112924448,
           93543278],
         [ 32409271, 120764301,   5680243,  ...,  24170641,  95132708,
           56714076]],

        [[ 51752204, 111493915, 130581677,  ...,  94512669,   9606886,
           44266644],
         [ 34838124,  76878025,   2307585,  ..., 100510751,  71217481,
          110731723]],

        [[ 65578592,  23817502, 106096809,  ..., 118350180,  19965836,
           64858278],
         [ 34404428,  12475758, 118260866,  ..., 126818329,  10023518,
          114571333]]], dtype=torch.int32)

In [ ]:
# add decomposition of s1* vec{g}
gs1 = gvector * s1

gs1

tensor([[      0,       0, 2097152,  ..., 2097152,       0, 2097152],
        [      0,       0,   32768,  ...,   32768,       0,   32768],
        [      0,       0,     512,  ...,     512,       0,     512],
        [      0,       0,       8,  ...,       8,       0,       8]])

In [ ]:
rlwepifft[:, 0, :] += gs1

In [ ]:
ksk = negacyclic_fft(rlwepifft, N, Q)

In [ ]:
ksk.size()

torch.Size([4, 2, 512])

In [ ]:
ct

tensor([[ 78261488,  48908637, 104842057,  ..., 124799417,  50596974,
         115122275],
        [ 29809111,  22452249,  54435983,  ..., 131073647, 100503685,
          81545739]], dtype=torch.int32)

In [ ]:
mdec = decrypt_from_fft(ctfft, s1fft)
print(mdec[:10])

tensor([33554434,        1,       -3, 33554428,       -3,       -1,        3,
              -3,       -6,        2], dtype=torch.int32)


In [ ]:
dct = decompose(ct[1])
dctfft = negacyclic_fft(dct, N, Q)
dctfft.size()

torch.Size([4, 512])

In [ ]:
print(ct[1])
print(torch.sum(dct*gvector, dim = 0))

tensor([ 29809111,  22452249,  54435983,  ..., 131073647, 100503685,
         81545739], dtype=torch.int32)
tensor([ 29809104,  22452248,  54435976,  ..., 131073640, 100503680,
         81545736])


Following is
$$
a \odot RLWE'_{s2}(s1) = RLWE_{s2}(a \cdot s1)
$$

In [ ]:
prodfft = dctfft.view(d, 1, N//2) * ksk
prodsumfft = torch.sum(prodfft, dim = 0)

prodsumfft.size(), prodsumfft

(torch.Size([2, 512]),
 tensor([[-3.5543e+15-2.1110e+13j, -4.2696e+14+4.5262e+11j,
          -7.4475e+13-5.3100e+12j,  ...,
          -2.5621e+13-2.7907e+11j, -4.9874e+13-6.9953e+12j,
          -1.5260e+14-1.2814e+13j],
         [-3.5234e+15-1.0319e+14j, -3.9655e+14-1.3726e+13j,
          -9.4416e+13-8.3850e+11j,  ...,
          -2.7011e+13+2.3127e+12j, -3.6247e+13-1.1700e+12j,
          -1.5074e+14+2.9692e+12j]], dtype=torch.complex128))

Adding $b$ to above, we get
$$
RLWE_{\boldsymbol{s2}}(\boldsymbol{a} \cdot \boldsymbol{s1}) + (\boldsymbol{b}, \boldsymbol{0}) = RLWE_{\boldsymbol{s2}}(\boldsymbol{a} \cdot \boldsymbol{s1} + \boldsymbol{b})
= RLWE_{\boldsymbol{s2}}(\boldsymbol{m} )
$$


In [ ]:
prodsumfft[0] += ctfft[0]

Now, we can decrypt using the switched key $s_2$.

Check if the decryption is successful.

In [ ]:
mks = decrypt_from_fft(prodsumfft, s2fft)
print("m:\n",m[:10])
print("decrypted:\n",mks[:10])
print("decrypted (scaled to 1):\n",mks[:10]/(Q//4))

m:
 tensor([33554432.,        0.,        0., 33554432.,        0.,        0.,
               0.,        0.,        0.,        0.])
decrypted:
 tensor([33841568,   291707,   361954, 33963323,   431422,   478976,   518779,
          556394,   623830,   634522], dtype=torch.int32)
decrypted (scaled to 1):
 tensor([1.0086, 0.0087, 0.0108, 1.0122, 0.0129, 0.0143, 0.0155, 0.0166, 0.0186,
        0.0189])
